## Food 11 Dataset

In [26]:
import os
import glob
import cv2

# Define the paths to the folders containing the images
eval_path = os.path.abspath('C:/Users/wdmoreno/OneDrive - Asesoftware S.A.S/Documentos/Food_data/evaluation')
train_path = os.path.abspath('C:/Users/wdmoreno/OneDrive - Asesoftware S.A.S/Documentos/Food_data/training')
val_path = os.path.abspath('C:/Users/wdmoreno/OneDrive - Asesoftware S.A.S/Documentos/Food_data/validation')

# Define a list of folder names
folders = ['Bread', 'Dairy product', 'Dessert', 'Egg', 'Fried food', 'Meat', 'Noodles-Pasta', 'Rice', 'Seafood', 'Soup', 'Vegetable-Fruit']

# Initialize arrays to store images from each directory
eval_images = []
train_images = []
val_images = []

# Loop over the folders
for folder in folders:
    # Read images from evaluation directory
    eval_folder = os.path.join(eval_path, folder)
    eval_images.extend([cv2.imread(img_path) for img_path in glob.glob(os.path.join(eval_folder, '*.jpg'))])
    
    # Read images from training directory
    train_folder = os.path.join(train_path, folder)
    train_images.extend([cv2.imread(img_path) for img_path in glob.glob(os.path.join(train_folder, '*.jpg'))])
    
    # Read images from validation directory
    val_folder = os.path.join(val_path, folder)
    val_images.extend([cv2.imread(img_path) for img_path in glob.glob(os.path.join(val_folder, '*.jpg'))])

# Print the number of images in each directory
print(f'Number of evaluation images: {len(eval_images)}')
print(f'Number of training images: {len(train_images)}')
print(f'Number of validation images: {len(val_images)}')



Number of evaluation images: 3347
Number of training images: 9860
Number of validation images: 3430


Analyze the data